In [1]:
#note that the adjacency matrix for a graph is a 0/1 matrix
#0/1 exists in any field, so we can view A \in (F_p)^{n^2} \cong F_{p^{n^2}} = F_q
#thus every adjacency matrix A can be viewed as an element of F_q

In [228]:
#determine if A is a 0/1 adjacency matrix with zeros on diagonal, symmetric
from sage.misc.flatten import flatten
def is_adj_matrix(A):
    #deal with case A is list of lists or matrix
    if type(A) == list:
        flat_A = flatten(A) #make the adjacency matrix into a list
        n = len(A) #get the number of columns of A
    else:
        flat_A = flatten([list(row) for row in A.rows()])
        n = A.ncols()
    zero_one = all(flat_A[k] == 0 or flat_A[k] == 1 for k in range(n^2)); print(zero_one)
    symmetric = matrix(A) == matrix(A).transpose(); print(symmetric)
    zero_diagonal = all(A[k][k] == 0 for k in range(n)); print(zero_diagonal)
    return zero_one and symmetric and zero_diagonal

In [229]:
def graph_embedding(A,p):
    try:
        assert p in Primes() #make sure p is prime
    except AssertionError:
        print("p must be prime")
        return
    if type(A) == list:
        n = len(A) #get the number of columns of A
        flat_A = flatten(A) #make the adjacency matrix into a list
    else:
        n = A.ncols()
        flat_A = flatten([list(row) for row in A.rows()])
    q = p**(n**2) #compute q=p^{n^2}
    alpha = GF(q).gen() #get a primitive element for the finite field F_q
    try:
        assert is_adj_matrix(A) #ensure A is an adjacency matrix
    except AssertionError:
        print("A must be a 0/1 adjacency matrix")
        return
    return sum([flat_A[k]*alpha**k for k in range(n^2)]) #let {1,alpha,...,alpha^{n^2-1}} be a basis and map the matrix into F_q

In [146]:
#naively find k such that alpha^k = x 
#where alpha is a generator for the finite field
def gen_power(x):
    alpha = x.parent().gen()
    k = 0
    while k < x.parent().order():
        if alpha**k == x:
            return k
        else:
            k += 1

In [142]:
graph_embedding([[0,1],[1,0]],3)

z4^2 + z4

In [143]:
gen_power(graph_embedding([[0,1],[1,0]],3))

29

In [211]:
#compute the integer associated to every adjacency matrix
n = 2
p = 3
all_zero_one_mats = [matrix(GF(3),A) for A in MatrixSpace(GF(2),n,n)]
[is_adj_matrix(A) for A in all_zero_one_mats]
#all_adj_mats = [A if ]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [230]:
is_adj_matrix(all_zero_one_mats[8])

True
True
True


True

In [223]:
zero_one = all(flat_A[k] == 0 or flat_A[k] == 1 for k in range(n^2))

In [227]:
flat_A[0]

[0, 1]